# Installations

In [ ]:
%%capture
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

In [ ]:
%%capture
!pip install -U datasets huggingface_hub fsspec

In [ ]:
%%capture
!pip install --upgrade --no-cache-dir unsloth

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported, unsloth_train
import torch
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
from datasets import load_dataset, DatasetDict
import wandb
from google.colab import userdata

# Initialisation

In [ ]:
hugging_face_token = userdata.get('HF_TOKEN')
wnb_token = userdata.get('WB_TOKEN')

# login to WnB
wandb.login(key=wnb_token)
run = wandb.init(
    project='Vikhr-Fine-tuning-1',
    job_type="training",
    anonymous="allow"
)

In [ ]:
max_seq_length = 4096 # the maximum sequence length a model can handle
dtype = None
load_in_4bit = True # 4bit quantization to reduce memory usage

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Vikhrmodels/Vikhr-Nemo-12B-Instruct-R-21-09-24",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Data Preparation

In [ ]:
# define a system prompt under prompt_style
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Ты следишь за качеством работы специалистов клиентской службы. Отвечай на русском языке.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = prompt_style.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = load_dataset("katarinaaaaa/evaluation-of-customer-service", split = "train", trust_remote_code=True)

In [ ]:
# get training and validation datasets
train_val_split = dataset.train_test_split(test_size=0.1, seed=123)
train_dataset = train_val_split['train']
validation_dataset = train_val_split['test']

train_dataset = train_dataset.map(formatting_prompts_func, batched = True,)
validation_dataset = validation_dataset.map(formatting_prompts_func, batched = True,)

# Train the model

In [ ]:
# apply LoRA (Low-Rank Adaptation) fine-tuning to the model
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # LoRA rank, determines the size of the trainable adapters (higher = more parameters, lower = more efficiency)
    target_modules=[  # list of transformer layers where LoRA adapters will be applied
        "q_proj",     # query projection in the self-attention mechanism
        "k_proj",     # key projection in the self-attention mechanism
        "v_proj",     # value projection in the self-attention mechanism
        "o_proj",     # output projection from the attention layer
        "gate_proj",  # used in feed-forward layers (MLP)
        "up_proj",    # part of the transformer’s feed-forward network (FFN)
        "down_proj",  # another part of the transformer’s FFN
    ],
    lora_alpha = 32,  # scaling factor for LoRA updates
    lora_dropout = 0, # dropout rate for LoRA layers
    bias = "none",    # specifies whether LoRA layers should learn bias terms
    use_gradient_checkpointing = "unsloth", # saves memory by recomputing activations instead of storing them
    random_state = 1234, # sets a seed for reproducibility, ensuring the same fine-tuning behavior across runs
    use_rslora = False,  # whether to use Rank-Stabilized LoRA
    loftq_config = None, # low-bit Fine-Tuning Quantization
)

In [ ]:
model.print_trainable_parameters()

In [ ]:
# initialize the fine-tuning trainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = validation_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2, # uses 2 CPU threads to speed up data preprocessing
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2, # number of examples processed per device (GPU) at a time
        gradient_accumulation_steps = 4, # accumulate gradients over 4 steps before updating weights
        warmup_steps = 5, # gradually increases learning rate for the first 5 steps
        num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(), # use FP16 if BF16 is not supported to speed up training
        bf16 = is_bfloat16_supported(), # use BF16 if supported (better numerical stability on newer GPUs)
        logging_steps = 10,
        optim = "adamw_8bit", # uses memory-efficient AdamW optimizer in 8-bit mode
        weight_decay = 0.01, # regularization to prevent overfitting
        lr_scheduler_type = "linear", # uses a linear learning rate schedule
        seed = 1234,
        output_dir = "outputs",
        gradient_checkpointing=True,
        do_eval=True,
        fp16_full_eval = True,
        per_device_eval_batch_size = 2,
        eval_accumulation_steps = 4,
        eval_strategy="steps",
        eval_steps=50,
        save_strategy="steps",
        save_steps=50,
        save_total_limit = 1,
    ),
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

# Training

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
# trainer_stats = trainer.train()
trainer_stats = unsloth_train(trainer)

In [ ]:
# save w&b statistics
wandb.finish()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# convert logs to dataframe
logs = trainer.state.log_history
df = pd.DataFrame(logs)
print(df)

# filter only rows with 'loss' key
df1 = df[['step', 'epoch', 'loss']].dropna()
df2 = df[['step','epoch', 'eval_loss']].dropna()

plt.figure(figsize=(8, 5))
plt.plot(df1['step'], df1['loss'], linestyle='-', label='Training Loss')
plt.plot(df2['step'], df2['eval_loss'], linestyle='-', label='Validation Loss')

plt.xlabel("Steps")
plt.ylabel("Loss")
plt.legend()
plt.grid()

# plt.show()
plt.savefig('my_plot1.png', dpi=600, transparent=True)

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

# Inference

In [ ]:
input = """Проанализируй предоставленный диалог между клиентом и оператором клиентской службы.
Оцени качество работы оператора от 0 до 10 баллов по каждому из следующих критериев:

1. Профессионализм и вежливость:
- Оператор использует корректность тона, отсутствие грубости или сарказма.
- Оператор соблюдает этикет (приветствие, прощание, обращение на «вы» и т.п.).
2. Соблюдение регламента и компетентность:
- Оператор представился и назвал имя компании.
- Оператор разбирается в своей области и корректно отвечает заданные на вопросы.
- Оператор заботится о положительном образе компании.
3. Эффективность коммуникации:
- Ответы оператора четкие и структурированные.
- Оператор умеет задавать уточняющие вопросы для понимания проблемы там, где они необходимы.
4. Решение проблемы:
- Оператор предоставляет корректную информацию.
- Оператор стремится предложить решение, даже если запрос сложный.
- Оператор предлагает альтернативы там, где это уместно.
- Предложенные оператором решения соответствуют общепринятым правилам и закону.
- Если обращение связано с ошибкой компании, оператор предлагает дополнительную компенсацию для сохранения лояльности клиента.
5. Грамотность речи:
- В сообщениях оператора отсутствуют грамматические и пунктуационные ошибки и опечатки.
6. Эмпатия и эмоциональный интеллект:
- Оператор учитывает эмоциональное состояние клиента (например, использует при необходимости извинения, слова поддержки).
- Оператор умеет снизить напряжение в конфликтной ситуации.

В ответе представь только json-файл со следующей структурой:

{
    "theme": "*Тема диалога*",
    "criteria":
    [
        {
            "criterion_name": "*Название критерия из списка выше*",
            "score": *Оценка указанного критерия (от 0 до 10). Если в диалоге отсутствуют данные для оценки критерия, поставь null*,
            "comments": "*Краткое резюме по данному критерию с примерами из диалога, иллюстрирующими выводы*",
        },
        {
            "criterion_name": "*Название критерия из списка выше*",
            "score": *Оценка указанного критерия (от 0 до 10). Если в диалоге отсутствуют данные для оценки критерия, поставь null*,
            "comments": "*Краткое резюме по данному критерию с примерами из диалога, иллюстрирующими выводы*",
        },
        ...
    ],
    "total_score": *Итоговая средняя оценка по всем указанным критериям*,
    "result": "*Решена ли проблема клиента (одно слово Да/Нет)*",
    "result_comment": "*Результат обращения клиента в одном предложении*",
    "recommendations": "*Краткие конкретные рекомендации оператору для повышения качества работы (если есть недочеты). Не используй пункты, пиши сплошным текстом.*",
}

Анализ должен быть объективным, без субъективных предположений. Не занижай оценку из-за незначительных недочетов.
Не добавляй в ответ ничего лишнего, начни ответ с {, закончи }. Используй только русский язык.

Диалог:

Оператор: Онлайн-кинотеатр «СинемаЛайф», Дарья. Чем могу помочь?
Клиент: Добрый день. Не могу продлить подписку — пишет «Ошибка платежа». Карта точно рабочая.
Оператор: Проверила ваш аккаунт: попытка оплаты 15 минут назад отклонена банком. Обновите данные карты в личном кабинете.
Клиент: Обновил — та же ошибка. Может, система глючит?
Оператор: Попробуйте сократить имя держателя карты: например, Ivanov I.I. вместо полного имени.
Клиент: О, сработало! Спасибо!
Оператор: Рада помочь! Иногда банки блокируют длинные названия. Подписка активна до 15 ноября.
Клиент: А если проблема повторится, куда писать?
Оператор: В этот чат или на support@cmnl.ru. Приложите скриншот ошибки.
Клиент: Хорошо. Ещё вопрос: почему пропали русские субтитры в новом сериале?
Оператор: Технический сбой. Исправим в течение суток. Извините за неудобство!
Клиент: Понял. Спасибо за оперативность!
Оператор: Всегда рады помочь! Приятного просмотра.
"""

FastLanguageModel.for_inference(model) # load the inference model using FastLanguageModel
inputs = tokenizer([prompt_style.format(input, "",)], return_tensors = "pt").to("cuda")

# use TextStreamer for real-time generating
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask,
                   streamer = text_streamer, pad_token_id = tokenizer.eos_token_id)

# Saving finetuned model

### Saving only LoRA adapters

In [ ]:
if False: model.save_pretrained("Vikhr-Customer-Service-Evaluation") # local saving
if False: tokenizer.save_pretrained("Vikhr-Customer-Service-Evaluation")
if False: model.push_to_hub("katarinaaaaa/Vikhr-Customer-Service-Evaluation", token=hugging_face_token) # online saving
if False: tokenizer.push_to_hub("katarinaaaaa/Vikhr-Customer-Service-Evaluation", token=hugging_face_token)

### Saving to float16 or int4

In [ ]:
# merge to 16bit
if False: model.save_pretrained_merged("Vikhr-Support-Evaluation-2", tokenizer, save_method = "merged_16bit") # local saving
model.push_to_hub_merged("katarinaaaaa/Vikhr-Customer-Service-Evaluation-2", tokenizer, save_method = "merged_16bit", token = hugging_face_token)

# merge to 4bit
if False: model.save_pretrained_merged("Vikhr-Customer-Service-Evaluation-2", tokenizer, save_method = "merged_4bit_forced",)
if False: model.push_to_hub_merged("katarinaaaaa/Vikhr-Customer-Service-Evaluation-2", tokenizer, save_method = "merged_4bit_forced", token = hugging_face_token)

# just LoRA adapters
if False: model.save_pretrained_merged("Vikhr-Customer-Service-Evaluation-2", tokenizer, save_method = "lora")
if False: model.push_to_hub_merged("katarinaaaaa/Vikhr-Customer-Service-Evaluation-2", tokenizer, save_method = "lora", token = hugging_face_token)

### GGUF conversion

In [ ]:
# save to 8bit Q8_0 - fast conversion, high resource use, but generally acceptable
if False: model.save_pretrained_gguf("Vikhr-Customer-Service-Evaluation-GGUF-2", tokenizer) # local saving
if False: model.push_to_hub_gguf("katarinaaaaa/Vikhr-Customer-Service-Evaluation-GGUF-2", tokenizer, token = hugging_face_token)

# save to 16bit GGUF
if False: model.save_pretrained_gguf("Vikhr-Customer-Service-Evaluation-GGUF-2", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("katarinaaaaa/Vikhr-Customer-Service-Evaluation-GGUF-2", tokenizer, quantization_method = "f16", token = hugging_face_token)

# save to q4_k_m GGUF - uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K
if False: model.save_pretrained_gguf("Vikhr-Customer-Service-Evaluation-GGUF-2", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("katarinaaaaa/Vikhr-Customer-Service-Evaluation-GGUF-2", tokenizer, quantization_method = "q4_k_m", token = hugging_face_token)

# save to q4_k_m GGUF - uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K
if False: model.save_pretrained_gguf("Vikhr-Customer-Service-Evaluation-GGUF-2", tokenizer, quantization_method = "q5_k_m")
if False: model.push_to_hub_gguf("katarinaaaaa/Vikhr-Customer-Service-Evaluation-GGUF-2", tokenizer, quantization_method = "q5_k_m", token = hugging_face_token)

# save to multiple GGUF options
model.push_to_hub_gguf(
    "katarinaaaaa/Vikhr-Customer-Service-Evaluation-GGUF-2",
    tokenizer,
    quantization_method = ["q8_0", "f16", "q4_k_m", "q5_k_m"],
    token = hugging_face_token,
)